In [1]:
import numpy as np

import matplotlib.pyplot as plt

import pandas as pd

from collections import defaultdict
from scipy.stats import kstest, norm, uniform, beta, expon, truncnorm

from scipy.optimize import brentq

from tqdm import tqdm
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import sdv
from sdv.metadata import SingleTableMetadata
from sdv.evaluation.single_table import evaluate_quality
from sdv.single_table import GaussianCopulaSynthesizer
tqdm.pandas()

import warnings
warnings.filterwarnings(action='ignore')
import seaborn as sns

In [2]:
data = pd.read_csv('American_Income.csv')

In [3]:
data_info = data.dtypes.to_dict()
num_column = []
category_column = []
for key in data_info.keys():
    if data_info[key] == 'int64':
        num_column.append(key)
    if data_info[key] == 'object':
        category_column.append(key)

In [4]:
## all functions

def findBestComp(data):
    bic = []
    n_components_range = range(1, 11)
    for n_components in n_components_range:
        gmm = GaussianMixture(n_components=n_components)
        gmm.fit(data.values.reshape(-1, 1))
        bic.append(gmm.bic(data.values.reshape(-1, 1)))

    optimal_components = n_components_range[np.argmin(bic)]

    return optimal_components


def gmm_cdf(x, gmm):
    """
    Compute the CDF of the GMM at point x.
    
    Parameters:
    - x: Point at which to evaluate the CDF
    - gmm: Fitted GaussianMixture object
    
    Returns:
    - CDF value at point x
    """
    cdf_value = 0
    for weight, mean, covariance in zip(gmm.weights_, gmm.means_, gmm.covariances_):
        cdf_value += weight * stats.norm.cdf(x, mean, np.sqrt(covariance)) 
    return cdf_value[0][0]


def gaussian_inverse_cdf(p_values):
    return norm.ppf(p_values)


def get_truncated_normal(mean=0, sd=1, low=0, upp=10):
    """ Return a truncated normal distribution. """
    return truncnorm((low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

def assign_intervals(categories):
    """ Assign intervals to categories based on their frequency. """
    freq = categories.value_counts(normalize=True)
    intervals = freq.cumsum()
    category_intervals = {}
    a = 0
    for category, cum_freq in intervals.items():
        b = cum_freq
        category_intervals[category] = (a, b)
        a = b
    return category_intervals

def sample_from_category(category, category_intervals):
    """ Sample a numerical value for a given category. """
    a, b = category_intervals[category]
    mean = (a + b) / 2
    sd = (b - a) / 6
    dist = get_truncated_normal(mean, sd, a, b)
    return dist.rvs()


def gmm_cdf_dict(x, gmm):
    """
    Compute the CDF of the GMM at point x using a dictionary to represent the GMM parameters.
    
    Parameters:
    - x: Point at which to evaluate the CDF
    - gmm: Dictionary with GMM parameters
    
    Returns:
    - CDF value at point x
    """
    cdf_value = 0
    for weight, mean, covariance in zip(gmm['weights_'], gmm['means_'], gmm['covariances_']):
        cdf_value += weight * norm.cdf(x, mean, np.sqrt(covariance))
    return cdf_value

def check_cdf_bounds(gmm, bounds, probabilities):
    results = []
    for p in probabilities:
        cdf_lower = gmm_cdf_dict(bounds[0], gmm)
        cdf_upper = gmm_cdf_dict(bounds[1], gmm)
        results.append((p, cdf_lower, cdf_upper, np.sign(cdf_lower - p), np.sign(cdf_upper - p)))
    return results


def compute_gmm_bounds(gmm, num_std=3):
    """
    Compute bounds for the GMM based on standard deviations from the mean.
    
    Parameters:
    - gmm: Dictionary with GMM parameters
    - num_std: Number of standard deviations to use for bounds
    
    Returns:
    - A tuple (lower_bound, upper_bound)
    """
    means = gmm['means_'].flatten()
    stds = np.sqrt(gmm['covariances_'].flatten())
    
    lower_bounds = means - num_std * stds
    upper_bounds = means + num_std * stds
    
    lower_bound = min(lower_bounds)
    upper_bound = max(upper_bounds)
    
    return lower_bound, upper_bound

def sample(F_inv, Sigma):
    """
    Sample numerical values from the distribution and covariances of the columns.
    
    Parameters:
    - F_inv: A list of inverse CDF functions for the marginals.
    - Sigma: The covariance matrix.
    
    Returns:
    - A sample vector x in the original space.
    """
    # Step 2: Random n-dimensional Gaussian vector
    n = Sigma.shape[0]
    v = np.random.randn(n)
    
    # Step 3: Cholesky decomposition
    L = np.linalg.cholesky(Sigma)
    
    # Step 4: u ← Lv
    u = L.dot(v)
    
    # Step 5: x ← [F_0^(-1)(Φ(u_0)), F_1^(-1)(Φ(u_1)), ..., F_n^(-1)(Φ(u_n))]
    x = [F_inv_i(norm.cdf(u_i)) for F_inv_i, u_i in zip(F_inv, u)]
    
    return x



In [5]:
data

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [6]:
category_column

[' workclass',
 ' education',
 ' marital-status',
 ' occupation',
 ' relationship',
 ' race',
 ' sex',
 ' native-country',
 ' salary']

In [7]:
num_column #eduaction_num belongs to category

['age',
 ' fnlwgt',
 ' education-num',
 ' capital-gain',
 ' capital-loss',
 ' hours-per-week']

In [9]:
age_components = findBestComp(data['age'])

fnlwgt_components = findBestComp(data[' fnlwgt'])

gain_components = findBestComp(data[' capital-gain'])

loss_components = findBestComp(data[' capital-loss'])

hours_components = findBestComp(data[' hours-per-week'])

In [11]:
#inital the gmm to fit
gmm_age = GaussianMixture(n_components=age_components)
gmm_age.fit(data['age'].values.reshape(-1, 1))

# Fit a GMM to the 'hours-per-week' column
gmm_hours = GaussianMixture(n_components=hours_components)  
gmm_hours.fit(data[' hours-per-week'].values.reshape(-1, 1))

gmm_fnlwgt = GaussianMixture(n_components=fnlwgt_components)
gmm_fnlwgt.fit(data[' fnlwgt'].values.reshape(-1, 1))

gmm_gain = GaussianMixture(n_components=gain_components)
gmm_gain.fit(data[' capital-gain'].values.reshape(-1, 1))

gmm_loss = GaussianMixture(n_components=loss_components)
gmm_loss.fit(data[' capital-loss'].values.reshape(-1, 1))

GaussianMixture(n_components=10)

In [13]:
categories_workclass = data[' workclass']
category_intervals_workclass = assign_intervals(categories_workclass)
numerical_values_workclass = categories_workclass.apply(lambda x: sample_from_category(x, category_intervals_workclass))

categories_education = data[' education']
category_intervals_education = assign_intervals(categories_education)
numerical_values_education = categories_education.apply(lambda x: sample_from_category(x, category_intervals_education))

categories_marital = data[' marital-status']
category_intervals_marital = assign_intervals(categories_marital)
numerical_values_marital = categories_marital.apply(lambda x: sample_from_category(x, category_intervals_marital))

categories_occupation = data[' occupation']
category_intervals_occupation = assign_intervals(categories_occupation)
numerical_values_occupation = categories_occupation.apply(lambda x: sample_from_category(x, category_intervals_occupation))

categories_relationship = data[' relationship']
category_intervals_relationship = assign_intervals(categories_relationship)
numerical_values_relationship = categories_relationship.apply(lambda x: sample_from_category(x, category_intervals_relationship))

categories_race = data[' race']
category_intervals_race = assign_intervals(categories_race)
numerical_values_race = categories_race.apply(lambda x: sample_from_category(x, category_intervals_race))

categories_sex = data[' sex']
category_intervals_sex = assign_intervals(categories_sex)
numerical_values_sex = categories_sex.apply(lambda x: sample_from_category(x, category_intervals_sex))

categories_country = data[' native-country']
category_intervals_country = assign_intervals(categories_country)
numerical_values_country = categories_country.apply(lambda x: sample_from_category(x, category_intervals_country))

categories_salary = data[' salary']
category_intervals_salary = assign_intervals(categories_salary)
numerical_values_salary = categories_salary.apply(lambda x: sample_from_category(x, category_intervals_salary))

In [14]:
data[' workclass'] = numerical_values_workclass
data[' education'] = numerical_values_education
data[' marital-status'] = numerical_values_marital
data[' occupation'] = numerical_values_occupation
data[' relationship'] = numerical_values_relationship
data[' race'] = numerical_values_race
data[' sex'] = numerical_values_sex
data[' native-country'] = numerical_values_country
data[' salary'] = numerical_values_salary

In [15]:
data

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,0.906194,77516,0.633112,13,0.607033,0.442110,0.529146,0.436993,0.346928,2174,0,40,0.695305,0.489264
1,50,0.732790,83311,0.643058,13,0.327116,0.299604,0.152220,0.333890,0.342774,0,0,13,0.554602,0.446327
2,38,0.409777,215646,0.166287,9,0.841400,0.891692,0.574400,0.543855,0.272486,0,0,40,0.520416,0.462242
3,53,0.520768,234721,0.822816,7,0.161458,0.897766,0.163139,0.881013,0.245039,0,0,40,0.381167,0.355028
4,28,0.409488,338409,0.630088,13,0.157326,0.042700,0.945183,0.935365,0.749773,0,0,40,0.959067,0.392157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,0.329630,257302,0.862439,12,0.200397,0.953115,0.940848,0.357642,0.851211,0,0,38,0.433700,0.480626
32557,40,0.589032,154374,0.185625,9,0.244058,0.721171,0.277247,0.686494,0.459192,0,0,40,0.260463,0.862169
32558,58,0.457204,151910,0.257307,9,0.975380,0.414197,0.905031,0.396496,0.865927,0,0,40,0.461434,0.590923
32559,22,0.337993,201490,0.174143,9,0.594160,0.413011,0.753158,0.540187,0.251806,0,0,20,0.323667,0.181488


In [16]:
workclass_components = findBestComp(data[' workclass'])

education_components = findBestComp(data[' education'])

marital_components = findBestComp(data[' marital-status'])

occupation_components = findBestComp(data[' occupation'])

relationship_components = findBestComp(data[' relationship'])

race_components = findBestComp(data[' race'])

sex_components = findBestComp(data[' sex'])

country_components = findBestComp(data[' native-country'])

salary_components = findBestComp(data[' salary'])

In [17]:
gmm_workclass = GaussianMixture(n_components=workclass_components)
gmm_workclass.fit(data[' workclass'].values.reshape(-1, 1))

gmm_education = GaussianMixture(n_components=education_components)
gmm_education.fit(data[' education'].values.reshape(-1, 1))

gmm_marital = GaussianMixture(n_components=marital_components)
gmm_marital.fit(data[' marital-status'].values.reshape(-1, 1))

gmm_occupation = GaussianMixture(n_components=occupation_components)
gmm_occupation.fit(data[' occupation'].values.reshape(-1, 1))

gmm_relationship = GaussianMixture(n_components= relationship_components)
gmm_relationship.fit(data[' relationship'].values.reshape(-1, 1))

gmm_race = GaussianMixture(n_components=race_components)
gmm_race.fit(data[' race'].values.reshape(-1, 1))

gmm_sex = GaussianMixture(n_components=sex_components)
gmm_sex.fit(data[' sex'].values.reshape(-1, 1))

gmm_country = GaussianMixture(n_components=country_components)
gmm_country.fit(data[' native-country'].values.reshape(-1, 1))

gmm_salary = GaussianMixture(n_components=salary_components)
gmm_salary.fit(data[' salary'].values.reshape(-1, 1))

GaussianMixture(n_components=2)

In [18]:
GMM_cdf_table = pd.DataFrame()
GMM_cdf_table['age'] = data['age'].apply(lambda x: gmm_cdf(x, gmm_age))
GMM_cdf_table[' workclass'] = data[' workclass'].apply(lambda x: gmm_cdf(x, gmm_workclass))
GMM_cdf_table[' fnlwgt'] = data[' fnlwgt'].apply(lambda x: gmm_cdf(x, gmm_fnlwgt))
GMM_cdf_table[' education'] = data[' education'].apply(lambda x: gmm_cdf(x, gmm_education))
GMM_cdf_table[' marital-status'] = data[' marital-status'].apply(lambda x: gmm_cdf(x, gmm_marital))
GMM_cdf_table[' occupation'] = data[' occupation'].apply(lambda x: gmm_cdf(x, gmm_occupation))
GMM_cdf_table[' relationship'] = data[' relationship'].apply(lambda x: gmm_cdf(x, gmm_relationship))
GMM_cdf_table[' race'] = data[' race'].apply(lambda x: gmm_cdf(x, gmm_race))
GMM_cdf_table[' sex'] = data[' sex'].apply(lambda x: gmm_cdf(x, gmm_sex))
GMM_cdf_table[' capital-gain'] = data[' capital-gain'].apply(lambda x: gmm_cdf(x, gmm_gain))
GMM_cdf_table[' capital-loss'] = data[' capital-loss'].apply(lambda x: gmm_cdf(x, gmm_loss))
GMM_cdf_table[' hours-per-week'] = data[' hours-per-week'].apply(lambda x: gmm_cdf(x, gmm_hours))
GMM_cdf_table[' native-country'] = data[' native-country'].apply(lambda x: gmm_cdf(x, gmm_country))
GMM_cdf_table[' salary'] = data[' salary'].apply(lambda x: gmm_cdf(x, gmm_salary))


In [22]:
GMM_invers_cdf_table = pd.DataFrame()

GMM_invers_cdf_table['age'] = GMM_cdf_table['age'].apply(lambda x: gaussian_inverse_cdf(x))
GMM_invers_cdf_table[' workclass'] = GMM_cdf_table[' workclass'].apply(lambda x: gaussian_inverse_cdf(x))
GMM_invers_cdf_table[' fnlwgt'] = GMM_cdf_table[' fnlwgt'].apply(lambda x: gaussian_inverse_cdf(x))
GMM_invers_cdf_table[' education'] = GMM_cdf_table[' education'].apply(lambda x: gaussian_inverse_cdf(x))
GMM_invers_cdf_table[' marital-status'] = GMM_cdf_table[' marital-status'].apply(lambda x: gaussian_inverse_cdf(x))
GMM_invers_cdf_table[' occupation'] = GMM_cdf_table[' occupation'].apply(lambda x: gaussian_inverse_cdf(x))
GMM_invers_cdf_table[' relationship'] = GMM_cdf_table[' relationship'].apply(lambda x: gaussian_inverse_cdf(x))
GMM_invers_cdf_table[' race'] = GMM_cdf_table[' race'].apply(lambda x: gaussian_inverse_cdf(x))
GMM_invers_cdf_table[' sex'] = GMM_cdf_table[' sex'].apply(lambda x: gaussian_inverse_cdf(x))
GMM_invers_cdf_table[' capital-gain'] = GMM_cdf_table[' capital-gain'].apply(lambda x: gaussian_inverse_cdf(x))
GMM_invers_cdf_table[' capital-loss'] = GMM_cdf_table[' capital-loss'].apply(lambda x: gaussian_inverse_cdf(x))
GMM_invers_cdf_table[' hours-per-week'] = GMM_cdf_table[' hours-per-week'].apply(lambda x: gaussian_inverse_cdf(x))
GMM_invers_cdf_table[' native-country'] = GMM_cdf_table[' native-country'].apply(lambda x: gaussian_inverse_cdf(x))
GMM_invers_cdf_table[' salary'] = GMM_cdf_table[' salary'].apply(lambda x: gaussian_inverse_cdf(x))


In [25]:
covariance_matrix = GMM_invers_cdf_table.cov()

In [28]:
gmm_age_dict = {
    'weights_': gmm_age.weights_, 
    'means_': gmm_age.means_, 
    'covariances_': gmm_age.covariances_ 
}
gmm_workclass_dict = {
    'weights_': gmm_workclass.weights_, 
    'means_': gmm_workclass.means_, 
    'covariances_': gmm_workclass.covariances_ 
}
gmm_fnlwgt_dict = {
    'weights_': gmm_fnlwgt.weights_, 
    'means_': gmm_fnlwgt.means_, 
    'covariances_': gmm_fnlwgt.covariances_ 
}
gmm_education_dict = {
    'weights_': gmm_education.weights_, 
    'means_': gmm_education.means_, 
    'covariances_': gmm_education.covariances_ 
}
gmm_marital_dict = {
    'weights_': gmm_marital.weights_, 
    'means_': gmm_marital.means_, 
    'covariances_': gmm_marital.covariances_ 
}
gmm_occupation_dict = {
    'weights_': gmm_occupation.weights_, 
    'means_': gmm_occupation.means_, 
    'covariances_': gmm_occupation.covariances_ 
}
gmm_relationship_dict = {
    'weights_': gmm_relationship.weights_, 
    'means_': gmm_relationship.means_, 
    'covariances_': gmm_relationship.covariances_ 
}
gmm_race_dict = {
    'weights_': gmm_race.weights_, 
    'means_': gmm_race.means_, 
    'covariances_': gmm_race.covariances_ 
}
gmm_sex_dict = {
    'weights_': gmm_sex.weights_, 
    'means_': gmm_sex.means_, 
    'covariances_': gmm_sex.covariances_ 
}
gmm_gain_dict = {
    'weights_': gmm_gain.weights_, 
    'means_': gmm_gain.means_, 
    'covariances_': gmm_gain.covariances_ 
}
gmm_loss_dict = {
    'weights_': gmm_loss.weights_, 
    'means_': gmm_loss.means_, 
    'covariances_': gmm_loss.covariances_ 
}
gmm_country_dict = {
    'weights_': gmm_country.weights_, 
    'means_': gmm_country.means_, 
    'covariances_': gmm_country.covariances_ 
}
gmm_salary_dict = {
    'weights_': gmm_salary.weights_, 
    'means_': gmm_salary.means_, 
    'covariances_': gmm_salary.covariances_ 
}

gmm_hours_dict = {
    'weights_': gmm_hours.weights_,
    'means_': gmm_hours.means_, 
    'covariances_': gmm_hours.covariances_  
}


age_bounds = (GMM_cdf_table['age'].min(), GMM_cdf_table['age'].max())
workclass_bounds = (GMM_cdf_table[' workclass'].min(), GMM_cdf_table[' workclass'].max())
fnlwgt_bounds = (GMM_cdf_table[' fnlwgt'].min(), GMM_cdf_table[' fnlwgt'].max())
education_bounds = (GMM_cdf_table[' education'].min(), GMM_cdf_table[' education'].max())
marital_bounds = (GMM_cdf_table[' marital-status'].min(), GMM_cdf_table[' marital-status'].max())
occupation_bounds = (GMM_cdf_table[' occupation'].min(), GMM_cdf_table[' occupation'].max())
relationship_bounds = (GMM_cdf_table[' relationship'].min(), GMM_cdf_table[' relationship'].max())
race_bounds = (GMM_cdf_table[' race'].min(), GMM_cdf_table[' race'].max())
sex_bounds = (GMM_cdf_table[' sex'].min(), GMM_cdf_table[' sex'].max())
gain_bounds = (GMM_cdf_table[' capital-gain'].min(), GMM_cdf_table[' capital-gain'].max())
loss_bounds = (GMM_cdf_table[' capital-loss'].min(), GMM_cdf_table[' capital-loss'].max())
country_bounds = (GMM_cdf_table[' native-country'].min(), GMM_cdf_table[' native-country'].max())
salary_bounds = (GMM_cdf_table[' salary'].min(), GMM_cdf_table[' salary'].max())
hours_bounds = (GMM_cdf_table[' hours-per-week'].min(), GMM_cdf_table[' hours-per-week'].max())   
probabilities = [0.1, 0.5, 0.9]

age_cdf_check = check_cdf_bounds(gmm_age_dict, age_bounds, probabilities)
workclass_cdf_check = check_cdf_bounds(gmm_workclass_dict, workclass_bounds, probabilities)
fnlwgt_cdf_check = check_cdf_bounds(gmm_fnlwgt_dict, fnlwgt_bounds, probabilities)
education_cdf_check = check_cdf_bounds(gmm_education_dict, education_bounds, probabilities)
marital_cdf_check = check_cdf_bounds(gmm_marital_dict, marital_bounds, probabilities)
occupation_cdf_check = check_cdf_bounds(gmm_occupation_dict, occupation_bounds, probabilities)
relationship_cdf_check = check_cdf_bounds(gmm_relationship_dict, relationship_bounds, probabilities)
race_cdf_check = check_cdf_bounds(gmm_race_dict, race_bounds, probabilities)
sex_cdf_check = check_cdf_bounds(gmm_sex_dict, sex_bounds, probabilities)
gain_cdf_check = check_cdf_bounds(gmm_gain_dict, gain_bounds, probabilities)
loss_cdf_check = check_cdf_bounds(gmm_loss_dict, loss_bounds, probabilities)
country_cdf_check = check_cdf_bounds(gmm_country_dict, country_bounds, probabilities)
salary_cdf_check = check_cdf_bounds(gmm_salary_dict, salary_bounds, probabilities)
hours_cdf_check = check_cdf_bounds(gmm_hours_dict, hours_bounds, probabilities)
# (age_cdf_check, hours_cdf_check)

In [29]:
new_age_bounds = compute_gmm_bounds(gmm_age_dict)
new_workclass_bounds = compute_gmm_bounds(gmm_workclass_dict)
new_fnlwgt_bounds = compute_gmm_bounds(gmm_fnlwgt_dict)
new_marital_bounds = compute_gmm_bounds(gmm_marital_dict)
new_education_bounds = compute_gmm_bounds(gmm_education_dict)
new_occupation_bounds = compute_gmm_bounds(gmm_occupation_dict)
new_relationship_bounds = compute_gmm_bounds(gmm_relationship_dict)
new_race_bounds = compute_gmm_bounds(gmm_race_dict)
new_sex_bounds = compute_gmm_bounds(gmm_sex_dict)
new_gain_bounds = compute_gmm_bounds(gmm_gain_dict)
new_loss_bounds = compute_gmm_bounds(gmm_loss_dict)
new_country_bounds = compute_gmm_bounds(gmm_country_dict)
new_salary_bounds = compute_gmm_bounds(gmm_salary_dict)
new_hours_bounds = compute_gmm_bounds(gmm_hours_dict)

# # Check the new CDF values at the computed bounds
# new_age_cdf_check = check_cdf_bounds(gmm_age_dict, new_age_bounds, probabilities)
# new_hours_cdf_check = check_cdf_bounds(gmm_hours_dict, new_hours_bounds, probabilities)

In [32]:
def inverse_cdf_gmm_age(p):
    lower_bound, upper_bound = new_age_bounds
    for _ in range(5):
        f_lower = gmm_cdf(lower_bound, gmm_age) - p
        f_upper = gmm_cdf(upper_bound, gmm_age) - p
        # Check if the signs are different
        if np.sign(f_lower) != np.sign(f_upper):
            try:
                return brentq(lambda x: gmm_cdf(x, gmm_age) - p, lower_bound, upper_bound, xtol=1e-6, rtol=1e-6)
            except ValueError as e:
                print(f"Failed to converge for p={p} with bounds ({lower_bound}, {upper_bound}): {e}")
                return np.nan  # or some other sentinel value indicating failure
        else:
            # Adjust the bounds
            lower_bound *= 0.9  # shrink the bounds
            upper_bound *= 1.1  # expand the bounds
    print(f"Failed to find valid bounds for p={p} after {5} retries")
    return np.nan  # or some other sentinel value indicating failure

def inverse_cdf_gmm_workclass(p):
    lower_bound, upper_bound = new_workclass_bounds
    for _ in range(5):
        f_lower = gmm_cdf(lower_bound, gmm_workclass) - p
        f_upper = gmm_cdf(upper_bound, gmm_workclass) - p
        # Check if the signs are different
        if np.sign(f_lower) != np.sign(f_upper):
            try:
                return brentq(lambda x: gmm_cdf(x, gmm_workclass) - p, lower_bound, upper_bound, xtol=1e-6, rtol=1e-6)
            except ValueError as e:
                print(f"Failed to converge for p={p} with bounds ({lower_bound}, {upper_bound}): {e}")
                return np.nan  # or some other sentinel value indicating failure
        else:
            # Adjust the bounds
            lower_bound *= 0.9  # shrink the bounds
            upper_bound *= 1.1  # expand the bounds
    print(f"Failed to find valid bounds for p={p} after {5} retries")
    return np.nan  # or some other sentinel value indicating failure

def inverse_cdf_gmm_fnlwgt(p):
    lower_bound, upper_bound = new_fnlwgt_bounds
    for _ in range(5):
        f_lower = gmm_cdf(lower_bound, gmm_fnlwgt) - p
        f_upper = gmm_cdf(upper_bound, gmm_fnlwgt) - p
        # Check if the signs are different
        if np.sign(f_lower) != np.sign(f_upper):
            try:
                return brentq(lambda x: gmm_cdf(x, gmm_fnlwgt) - p, lower_bound, upper_bound, xtol=1e-6, rtol=1e-6)
            except ValueError as e:
                print(f"Failed to converge for p={p} with bounds ({lower_bound}, {upper_bound}): {e}")
                return np.nan  # or some other sentinel value indicating failure
        else:
            # Adjust the bounds
            lower_bound *= 0.9  # shrink the bounds
            upper_bound *= 1.1  # expand the bounds
    print(f"Failed to find valid bounds for p={p} after {5} retries")
    return np.nan  # or some other sentinel value indicating failure

def inverse_cdf_gmm_education(p):
    lower_bound, upper_bound = new_education_bounds
    for _ in range(5):
        f_lower = gmm_cdf(lower_bound, gmm_education) - p
        f_upper = gmm_cdf(upper_bound, gmm_education) - p
        # Check if the signs are different
        if np.sign(f_lower) != np.sign(f_upper):
            try:
                return brentq(lambda x: gmm_cdf(x, gmm_education) - p, lower_bound, upper_bound, xtol=1e-6, rtol=1e-6)
            except ValueError as e:
                print(f"Failed to converge for p={p} with bounds ({lower_bound}, {upper_bound}): {e}")
                return np.nan  # or some other sentinel value indicating failure
        else:
            # Adjust the bounds
            lower_bound *= 0.9  # shrink the bounds
            upper_bound *= 1.1  # expand the bounds
    print(f"Failed to find valid bounds for p={p} after {5} retries")
    return np.nan  # or some other sentinel value indicating failure


def inverse_cdf_gmm_marital(p):
    lower_bound, upper_bound = new_marital_bounds
    for _ in range(5):
        f_lower = gmm_cdf(lower_bound, gmm_marital) - p
        f_upper = gmm_cdf(upper_bound, gmm_marital) - p
        # Check if the signs are different
        if np.sign(f_lower) != np.sign(f_upper):
            try:
                return brentq(lambda x: gmm_cdf(x, gmm_marital) - p, lower_bound, upper_bound, xtol=1e-6, rtol=1e-6)
            except ValueError as e:
                print(f"Failed to converge for p={p} with bounds ({lower_bound}, {upper_bound}): {e}")
                return np.nan  # or some other sentinel value indicating failure
        else:
            # Adjust the bounds
            lower_bound *= 0.9  # shrink the bounds
            upper_bound *= 1.1  # expand the bounds
    print(f"Failed to find valid bounds for p={p} after {5} retries")
    return np.nan  # or some other sentinel value indicating failure


def inverse_cdf_gmm_occupation(p):
    lower_bound, upper_bound = new_occupation_bounds
    for _ in range(5):
        f_lower = gmm_cdf(lower_bound, gmm_occupation) - p
        f_upper = gmm_cdf(upper_bound, gmm_occupation) - p
        # Check if the signs are different
        if np.sign(f_lower) != np.sign(f_upper):
            try:
                return brentq(lambda x: gmm_cdf(x, gmm_occupation) - p, lower_bound, upper_bound, xtol=1e-6, rtol=1e-6)
            except ValueError as e:
                print(f"Failed to converge for p={p} with bounds ({lower_bound}, {upper_bound}): {e}")
                return np.nan  # or some other sentinel value indicating failure
        else:
            # Adjust the bounds
            lower_bound *= 0.9  # shrink the bounds
            upper_bound *= 1.1  # expand the bounds
    print(f"Failed to find valid bounds for p={p} after {5} retries")
    return np.nan  # or some other sentinel value indicating failure


def inverse_cdf_gmm_relationship(p):
    lower_bound, upper_bound = new_relationship_bounds
    for _ in range(5):
        f_lower = gmm_cdf(lower_bound, gmm_relationship) - p
        f_upper = gmm_cdf(upper_bound, gmm_relationship) - p
        # Check if the signs are different
        if np.sign(f_lower) != np.sign(f_upper):
            try:
                return brentq(lambda x: gmm_cdf(x, gmm_relationship) - p, lower_bound, upper_bound, xtol=1e-6, rtol=1e-6)
            except ValueError as e:
                print(f"Failed to converge for p={p} with bounds ({lower_bound}, {upper_bound}): {e}")
                return np.nan  # or some other sentinel value indicating failure
        else:
            # Adjust the bounds
            lower_bound *= 0.9  # shrink the bounds
            upper_bound *= 1.1  # expand the bounds
    print(f"Failed to find valid bounds for p={p} after {5} retries")
    return np.nan  # or some other sentinel value indicating failure


def inverse_cdf_gmm_race(p):
    lower_bound, upper_bound = new_race_bounds
    for _ in range(5):
        f_lower = gmm_cdf(lower_bound, gmm_race) - p
        f_upper = gmm_cdf(upper_bound, gmm_race) - p
        # Check if the signs are different
        if np.sign(f_lower) != np.sign(f_upper):
            try:
                return brentq(lambda x: gmm_cdf(x, gmm_race) - p, lower_bound, upper_bound, xtol=1e-6, rtol=1e-6)
            except ValueError as e:
                print(f"Failed to converge for p={p} with bounds ({lower_bound}, {upper_bound}): {e}")
                return np.nan  # or some other sentinel value indicating failure
        else:
            # Adjust the bounds
            lower_bound *= 0.9  # shrink the bounds
            upper_bound *= 1.1  # expand the bounds
    print(f"Failed to find valid bounds for p={p} after {5} retries")
    return np.nan  # or some other sentinel value indicating failure


def inverse_cdf_gmm_sex(p):
    lower_bound, upper_bound = new_sex_bounds
    for _ in range(5):
        f_lower = gmm_cdf(lower_bound, gmm_sex) - p
        f_upper = gmm_cdf(upper_bound, gmm_sex) - p
        # Check if the signs are different
        if np.sign(f_lower) != np.sign(f_upper):
            try:
                return brentq(lambda x: gmm_cdf(x, gmm_sex) - p, lower_bound, upper_bound, xtol=1e-6, rtol=1e-6)
            except ValueError as e:
                print(f"Failed to converge for p={p} with bounds ({lower_bound}, {upper_bound}): {e}")
                return np.nan  # or some other sentinel value indicating failure
        else:
            # Adjust the bounds
            lower_bound *= 0.9  # shrink the bounds
            upper_bound *= 1.1  # expand the bounds
    print(f"Failed to find valid bounds for p={p} after {5} retries")
    return np.nan  # or some other sentinel value indicating failure


def inverse_cdf_gmm_gain(p):
    lower_bound, upper_bound = new_gain_bounds
    for _ in range(5):
        f_lower = gmm_cdf(lower_bound, gmm_gain) - p
        f_upper = gmm_cdf(upper_bound, gmm_gain) - p
        # Check if the signs are different
        if np.sign(f_lower) != np.sign(f_upper):
            try:
                return brentq(lambda x: gmm_cdf(x, gmm_gain) - p, lower_bound, upper_bound, xtol=1e-6, rtol=1e-6)
            except ValueError as e:
                print(f"Failed to converge for p={p} with bounds ({lower_bound}, {upper_bound}): {e}")
                return np.nan  # or some other sentinel value indicating failure
        else:
            # Adjust the bounds
            lower_bound *= 0.9  # shrink the bounds
            upper_bound *= 1.1  # expand the bounds
    print(f"Failed to find valid bounds for p={p} after {5} retries")
    return np.nan  # or some other sentinel value indicating failure



def inverse_cdf_gmm_loss(p):
    lower_bound, upper_bound = new_loss_bounds
    for _ in range(5):
        f_lower = gmm_cdf(lower_bound, gmm_loss) - p
        f_upper = gmm_cdf(upper_bound, gmm_loss) - p
        # Check if the signs are different
        if np.sign(f_lower) != np.sign(f_upper):
            try:
                return brentq(lambda x: gmm_cdf(x, gmm_loss) - p, lower_bound, upper_bound, xtol=1e-6, rtol=1e-6)
            except ValueError as e:
                print(f"Failed to converge for p={p} with bounds ({lower_bound}, {upper_bound}): {e}")
                return np.nan  # or some other sentinel value indicating failure
        else:
            # Adjust the bounds
            lower_bound *= 0.9  # shrink the bounds
            upper_bound *= 1.1  # expand the bounds
    print(f"Failed to find valid bounds for p={p} after {5} retries")
    return np.nan  # or some other sentinel value indicating failure

def inverse_cdf_gmm_hours(p):
    lower_bound, upper_bound = new_hours_bounds
    for _ in range(5):
        f_lower = gmm_cdf(lower_bound, gmm_hours) - p
        f_upper = gmm_cdf(upper_bound, gmm_hours) - p
        if np.sign(f_lower) != np.sign(f_upper):
            try:
                return brentq(lambda x: gmm_cdf(x, gmm_hours) - p, lower_bound, upper_bound, xtol=1e-6, rtol=1e-6)
            except ValueError as e:
                print(f"Failed to converge for p={p} with bounds ({lower_bound}, {upper_bound}): {e}")
                return np.nan  # or some other sentinel value indicating failure
        else:
            lower_bound *= 0.9  # shrink the bounds
            upper_bound *= 1.1  # expand the bounds
    print(f"Failed to find valid bounds for p={p} after {5} retries")
    return np.nan  # or some other sentinel value indicating failure

def inverse_cdf_gmm_country(p):
    lower_bound, upper_bound = new_country_bounds
    for _ in range(5):
        f_lower = gmm_cdf(lower_bound, gmm_country) - p
        f_upper = gmm_cdf(upper_bound, gmm_country) - p
        # Check if the signs are different
        if np.sign(f_lower) != np.sign(f_upper):
            try:
                return brentq(lambda x: gmm_cdf(x, gmm_country) - p, lower_bound, upper_bound, xtol=1e-6, rtol=1e-6)
            except ValueError as e:
                print(f"Failed to converge for p={p} with bounds ({lower_bound}, {upper_bound}): {e}")
                return np.nan  # or some other sentinel value indicating failure
        else:
            # Adjust the bounds
            lower_bound *= 0.9  # shrink the bounds
            upper_bound *= 1.1  # expand the bounds
    print(f"Failed to find valid bounds for p={p} after {5} retries")
    return np.nan  # or some other sentinel value indicating failure


def inverse_cdf_gmm_salary(p):
    lower_bound, upper_bound = new_salary_bounds
    for _ in range(5):
        f_lower = gmm_cdf(lower_bound, gmm_salary) - p
        f_upper = gmm_cdf(upper_bound, gmm_salary) - p
        # Check if the signs are different
        if np.sign(f_lower) != np.sign(f_upper):
            try:
                return brentq(lambda x: gmm_cdf(x, gmm_salary) - p, lower_bound, upper_bound, xtol=1e-6, rtol=1e-6)
            except ValueError as e:
                print(f"Failed to converge for p={p} with bounds ({lower_bound}, {upper_bound}): {e}")
                return np.nan  # or some other sentinel value indicating failure
        else:
            # Adjust the bounds
            lower_bound *= 0.9  # shrink the bounds
            upper_bound *= 1.1  # expand the bounds
    print(f"Failed to find valid bounds for p={p} after {5} retries")
    return np.nan  # or some other sentinel value indicating failure


F_inv = [inverse_cdf_gmm_age, inverse_cdf_gmm_workclass, inverse_cdf_gmm_fnlwgt, inverse_cdf_gmm_education, inverse_cdf_gmm_marital, inverse_cdf_gmm_occupation, 
         inverse_cdf_gmm_relationship, inverse_cdf_gmm_race, inverse_cdf_gmm_sex, inverse_cdf_gmm_gain, inverse_cdf_gmm_loss, inverse_cdf_gmm_hours, inverse_cdf_gmm_country, 
         inverse_cdf_gmm_salary]

In [31]:
num_rows = data.shape[0]  # Number of rows you want to generate
synthetic_dataset = [sample(F_inv, covariance_matrix) for _ in tqdm(range(num_rows))]
synthetic_df_full = pd.DataFrame(synthetic_dataset, columns=['age', ' workclass', ' fnlwgt', ' education', ' marital-status', ' occupation', ' relationship', ' race',
                                                             ' sex', ' capital-gain', ' capital-loss', ' hours-per-week', ' native-country', ' salary'])

  1%|          | 246/32561 [00:17<37:03, 14.53it/s]

Failed to find valid bounds for p=0.0007253601583202063 after 5 retries


  2%|▏         | 688/32561 [00:48<37:52, 14.02it/s]

Failed to find valid bounds for p=4.1729042973430085e-05 after 5 retries


  4%|▎         | 1200/32561 [01:24<38:45, 13.49it/s]

...

 99%|█████████▉| 32296/32561 [37:59<00:18, 14.28it/s]

Failed to find valid bounds for p=0.0002468406735222498 after 5 retries


100%|█████████▉| 32412/32561 [38:07<00:10, 13.77it/s]

Failed to find valid bounds for p=9.02682897938149e-05 after 5 retries


100%|██████████| 32561/32561 [38:18<00:00, 14.17it/s]


In [62]:
synthetic_df_full.isnull().sum()

age                 0
 workclass          5
 fnlwgt             0
 education          5
 marital-status    13
 occupation         2
 relationship       8
 race               4
 sex               42
 capital-gain       0
 capital-loss       0
 hours-per-week     8
 native-country    41
 salary            34
dtype: int64

In [38]:
synthetic_df_full.to_csv('synthetic_data_full.csv')

In [63]:
column_mean = synthetic_df_full.mean()

synthetic_df_full_filled = synthetic_df_full.fillna(column_mean)

In [64]:
synthetic_df_full_filled

,age,workclass,fnlwgt,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,47.602875,0.856669,296566.030725,0.984768,0.233210,0.649474,0.222499,0.409044,0.816470,0.000527,0.000145,40.001350,0.369657,0.809970
1,32.574126,0.420624,59019.315568,0.759543,0.332804,0.065237,0.850943,0.605872,0.738007,-0.001290,-0.000386,24.542558,0.384696,0.136709
2,41.354508,0.801740,246596.280778,0.932366,0.641960,0.206858,0.561668,0.357424,0.395557,-0.000174,0.000599,40.000271,0.391150,0.843468
3,19.452579,0.795551,177906.366159,0.731019,0.618765,0.743068,0.549303,0.189635,0.912975,0.000118,-0.000587,39.999642,0.276982,0.798659
4,51.995030,0.731495,108521.661446,0.826903,0.114574,0.540635,0.036591,0.584521,0.073733,-0.000094,0.000276,67.932632,0.418206,0.479894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,30.505639,0.368888,76959.644739,0.412932,0.871281,0.884887,0.726212,0.643755,0.454662,-0.000152,-0.000032,39.998362,0.130690,0.162942
32557,17.055637,0.144870,420215.922994,0.067869,0.183269,0.052438,0.865840,0.863900,0.583839,0.000706,0.000365,46.752890,0.626080,0.787263
32558,36.403838,0.661343,46522.091577,0.459857,0.850907,0.678214,0.230242,0.476655,0.267591,-0.000971,0.000149,40.001383,0.371916,0.128849
32559,23.737994,0.278785,633733.796930,0.477951,0.866243,0.800750,0.876695,0.908915,0.525720,-0.000374,0.000621,40.001594,0.230323,0.507307


In [65]:
def numerical_to_category(num_value, category_intervals):
    """ Convert a numerical value back to its corresponding category. """
    for category, (a, b) in category_intervals.items():
        if a <= num_value < b:
            return category
        elif num_value > 1 and round(b) == 1:
            return category
        elif num_value < 0 and round(b) == 0:
            return category

In [66]:
inversed_categories_workclass = synthetic_df_full_filled[' workclass'].apply(lambda x: numerical_to_category(x, category_intervals_workclass))
inversed_categories_education = synthetic_df_full_filled[' education'].apply(lambda x: numerical_to_category(x, category_intervals_education))
inversed_categories_marital = synthetic_df_full_filled[' marital-status'].apply(lambda x: numerical_to_category(x, category_intervals_marital))
inversed_categories_occupation = synthetic_df_full_filled[' occupation'].apply(lambda x: numerical_to_category(x, category_intervals_occupation))
inversed_categories_relationship = synthetic_df_full_filled[' relationship'].apply(lambda x: numerical_to_category(x, category_intervals_relationship))
inversed_categories_race = synthetic_df_full_filled[' race'].apply(lambda x: numerical_to_category(x, category_intervals_race))
inversed_categories_sex = synthetic_df_full_filled[' sex'].apply(lambda x: numerical_to_category(x, category_intervals_sex))
inversed_categories_country = synthetic_df_full_filled[' native-country'].apply(lambda x: numerical_to_category(x, category_intervals_country))
inversed_categories_salary = synthetic_df_full_filled[' salary'].apply(lambda x: numerical_to_category(x, category_intervals_salary))

In [67]:
synthetic_df_full_filled[' workclass'] = inversed_categories_workclass

synthetic_df_full_filled[' education'] = inversed_categories_education

synthetic_df_full_filled[' marital-status'] = inversed_categories_marital

synthetic_df_full_filled[' occupation'] = inversed_categories_occupation

synthetic_df_full_filled[' relationship'] = inversed_categories_relationship

synthetic_df_full_filled[' race'] = inversed_categories_race

synthetic_df_full_filled[' sex'] = inversed_categories_sex

synthetic_df_full_filled[' native-country'] = inversed_categories_country

synthetic_df_full_filled[' salary'] = inversed_categories_salary

In [68]:
synthetic_df_full_filled

,age,workclass,fnlwgt,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,47.602875,?,296566.030725,5th-6th,Married-civ-spouse,Other-service,Husband,White,Female,0.000527,0.000145,40.001350,United-States,>50K
1,32.574126,Private,59019.315568,Masters,Married-civ-spouse,Prof-specialty,Unmarried,White,Female,-0.001290,-0.000386,24.542558,United-States,<=50K
2,41.354508,Local-gov,246596.280778,Prof-school,Never-married,Craft-repair,Not-in-family,White,Male,-0.000174,0.000599,40.000271,United-States,>50K
3,19.452579,Local-gov,177906.366159,Masters,Never-married,Machine-op-inspct,Not-in-family,White,Female,0.000118,-0.000587,39.999642,United-States,>50K
4,51.995030,Self-emp-not-inc,108521.661446,11th,Married-civ-spouse,Sales,Husband,White,Male,-0.000094,0.000276,67.932632,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,30.505639,Private,76959.644739,Some-college,Divorced,Handlers-cleaners,Own-child,White,Male,-0.000152,-0.000032,39.998362,United-States,<=50K
32557,17.055637,Private,420215.922994,HS-grad,Married-civ-spouse,Prof-specialty,Unmarried,Black,Male,0.000706,0.000365,46.752890,United-States,>50K
32558,36.403838,Private,46522.091577,Some-college,Divorced,Other-service,Husband,White,Male,-0.000971,0.000149,40.001383,United-States,<=50K
32559,23.737994,Private,633733.796930,Some-college,Divorced,?,Unmarried,Black,Male,-0.000374,0.000621,40.001594,United-States,<=50K


In [69]:
data1 = pd.read_csv("American_Income.csv")
data_for_test = data1.drop(columns=[' education-num'])

data_for_test

,age,workclass,fnlwgt,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [70]:
metadata1 = SingleTableMetadata()
metadata1.detect_from_dataframe(data = data_for_test)
baseline_quality_report = evaluate_quality(
    data_for_test,  #real data
    synthetic_df_full_filled, #synthetic data(perfect synthesizer generated(holdout))
    metadata1
)

Creating report: 100%|██████████| 4/4 [00:00<00:00,  5.14it/s]



Overall Quality Score: 87.67%

Properties:
Column Shapes: 90.2%
Column Pair Trends: 85.14%


In [57]:
metadata1 = SingleTableMetadata()

metadata1.detect_from_dataframe(data = data_for_test)
synthesier = GaussianCopulaSynthesizer(metadata1)
synthesier.fit(data_for_test)
synthetic_data = synthesier.sample(num_rows = num_rows)  #keep the number of rows equal to the data hold
synthetic_data 

,age,workclass,fnlwgt,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,44,Private,137738,HS-grad,Divorced,Transport-moving,Own-child,White,Female,87580,23,55,United-States,>50K
1,33,Federal-gov,144753,11th,Married-civ-spouse,Exec-managerial,Unmarried,White,Female,5255,0,60,United-States,<=50K
2,22,Private,263046,Prof-school,Married-civ-spouse,Transport-moving,Husband,White,Male,51338,0,49,United-States,>50K
3,47,Local-gov,142107,9th,Divorced,Prof-specialty,Own-child,Black,Female,49060,0,32,United-States,<=50K
4,24,Federal-gov,342515,Bachelors,Never-married,Transport-moving,Unmarried,Black,Male,3978,62,39,India,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,42,Federal-gov,74028,Masters,Married-spouse-absent,Transport-moving,Unmarried,White,Female,99592,0,38,United-States,<=50K
32557,21,Private,142426,Assoc-acdm,Never-married,Exec-managerial,Not-in-family,Black,Female,536,0,33,United-States,<=50K
32558,33,State-gov,106801,Masters,Never-married,Handlers-cleaners,Husband,White,Female,26961,0,24,United-States,<=50K
32559,49,?,148553,7th-8th,Married-civ-spouse,?,Own-child,White,Male,62266,112,39,Cuba,>50K


In [59]:
metadata1

{
    "columns": {
        "age": {
            "sdtype": "numerical"
        },
        " workclass": {
            "sdtype": "categorical"
        },
        " fnlwgt": {
            "sdtype": "numerical"
        },
        " education": {
            "sdtype": "categorical"
        },
        " marital-status": {
            "sdtype": "categorical"
        },
        " occupation": {
            "sdtype": "categorical"
        },
        " relationship": {
            "sdtype": "categorical"
        },
        " race": {
            "sdtype": "categorical"
        },
        " sex": {
            "sdtype": "categorical"
        },
        " capital-gain": {
            "sdtype": "numerical"
        },
        " capital-loss": {
            "sdtype": "numerical"
        },
        " hours-per-week": {
            "sdtype": "numerical"
        },
        " native-country": {
            "sdtype": "categorical"
        },
        " salary": {
            "sdtype": "categorical"
       

In [58]:
metadata1 = SingleTableMetadata()
metadata1.detect_from_dataframe(data = data_for_test)
baseline_quality_report = evaluate_quality(
    data_for_test,  #real data
    synthetic_data, #synthetic data(perfect synthesizer generated(holdout))
    metadata1
)

Creating report: 100%|██████████| 4/4 [00:00<00:00,  4.01it/s]



Overall Quality Score: 71.81%

Properties:
Column Shapes: 74.55%
Column Pair Trends: 69.07%
